# Google BigQuery
### Sample Upload a list to GBQ, join to GBQ Data, Download results
David Andrews
Legal Analytics

This sample connects to Google BigQuery to retrieve patent data.  You need to do some setup in Google's cloud dev platform if you are not already using Google BigQuery.  You can reference one of the many great tutorials on setting up Google BigQuery for the first time to get the libraries installed, credentials cached, and project ID setup.

#### Caveat
As-is.  Use at your own risk.  Joining data can be hard and you might think you have the right results, but double-check before you rely on this data, and check with other data sources before you litigate!

In [1]:
#import lots of stuff
from google.cloud import bigquery
import pandas as pd
from oauth2client.client import GoogleCredentials
from googleapiclient import discovery
from IPython.display import display, HTML

# Upload data to GBQ

In [2]:
#load in the patents we want to join
#these files are just flat lists of patent numbers.  You can load locally using a local filepath,e.g. r"c:\data\portfolio.csv"
#use read_excel if you have an xlsx file instead of read_csv.  read_excel doesn't accept a url, so it has to be local.
example_url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTwNjPYeJV6l0lOTjMnI65rE4i_Prtc4Gnku3HupqBzuZ5v9wzhYWAA26AivTkFPw_AbwGuiuqoj_lq/pub?output=csv"
df_patent_numbers = pd.read_csv(example_url)

#patent numbers come in lots of different flavors, like "US9000000B2" "9000000" "US9000000" "US-9000000-B2" or "9,000,000" 
#you can use RegEx101.com to try out regular expressions to match your format and reformat to "US-9000000-B2" to match GBQ
#Sample format is US1234567B2 so we change it to US-1234567-B2
df_patent_numbers['Publication_Number'] = df_patent_numbers['Publication_Number'].str.replace(r"(\D*)(\d*)(\D\d?)", "\\1-\\2-\\3")
#Application numbers are wierd.  Sometimes they have an extra zero at the 5th character into the number.  We need to do
#this for the sample data that was downloaded from Google Patents.
#need to strip it so it matches the format in Google BigQuery
df_patent_numbers['Publication_Number'] = df_patent_numbers['Publication_Number'].str.replace(r"(\D*-)(\d{4})0(\d{6})(-A1)",
                                                                                         "\\1\\2\\3\\4")
#if the Regular expressions in the replace function are challenging to understand, try plugging them into 
#regex101.com along with some sample numbers and using the explanation to understand what each portion is doing.
#unpacking the above REGEXP a little:
# (\D*) = grab all the non-digits at the front e.g. US
# (\d*) = grab all the digits, e.g. 1234567
# (\D\d?) = grab a trailing non-digit optionally followed by a digit e.g. A1 or B2 or A 

#drop dulicates.  Sometimes duplicates can cause issues in joins.
df_patent_numbers.drop_duplicates(['Publication_Number'], inplace=True)
#show the training set to see what it looks like.  Make sure it imported correctly.
display(df_patent_numbers.head())


,Publication_Number
0,US-2003004827-A1
1,US-2003097331-A1
2,US-2003191719-A1
3,US-2004123129-A1
4,US-2005177716-A1


# Connect to  Google BigQuery and upload patent numbers

In [3]:
#load the dataset to Google BigQuery so we can join against the public patents data

# Variables to be used to access GBQ, replace with your project id, and optionally change the table and
# and dataset name.
PROJECT_ID = 'patenttest-182300' #change this to your project ID
DEST_DATASET = 'my_new_dataset'
table_name = 'patent_numbers_temp'

# Create a python client we can use for executing table creation queries
client = bigquery.Client(project=PROJECT_ID)
# Create an HTTP client for additional functionality.
credentials = GoogleCredentials.get_application_default()
http_client = discovery.build('bigquery', 'v2', credentials=credentials)

#attach to the dataset
dataset = client.dataset(DEST_DATASET)

#create the table by having Pandas push up the dataframe as a table
full_table_path = '{}.{}'.format(DEST_DATASET, table_name)
print("Inserting", len(df_patent_numbers), "rows into:",full_table_path)
df_patent_numbers.to_gbq(destination_table=full_table_path,
          project_id=PROJECT_ID,
          if_exists='replace')

Inserting 791 rows into: my_new_dataset.patent_numbers_temp



Streaming Insert is 100.0% Complete




# Query BigQuery to get training text

In [4]:
#create our query string.  The query joins the BigQuery patent data using our training set on
# publication number.  Because we formatted in Python to match the BigQuery format, we can just use equality
# in the "on" clause of the join, but you could also do manipulations here to the format of the publication number
# The query just adds the title and grant date.
query = """
select pubs.publication_number, 
  (SELECT text from UNNEST(pubs.title_localized) LIMIT 1) as title,
  pubs.grant_date
from
   `patents-public-data.patents.publications`  as pubs, UNNEST(title_localized) as title
  JOIN `""" + full_table_path + """`  as input
  on pubs.publication_number = input.Publication_Number
"""
df_patent_data = pd.read_gbq(query, project_id=PROJECT_ID, dialect='standard')
#check to make sure we got back a dataset that looks right.
display(df_patent_data.head(10))



Requesting query... ok.
Job ID: job_uQs6PF_CPEuOLP6nVAl5qGTNUEm6
Query running...
Query done.
Processed: 8.7 GB
Standard price: $0.04 USD

Retrieving results...
Got 791 rows.

Total time taken 2.9 s.
Finished at 2018-01-20 18:23:05.


,publication_number,title,grant_date
0,US-2006218651-A1,"Trusted infrastructure support systems, method...",0
1,US-2004123129-A1,"Trusted infrastructure support systems, method...",0
2,US-2003097331-A1,Systems for financial and electronic commerce,0
3,US-2003004827-A1,Payment system,0
4,US-2005197957-A1,Parcel manager for distributed electronic bill...,0
5,US-8140567-B2,Measuring entity extraction complexity,20120320
6,US-7734631-B2,Associating information with an electronic doc...,20100608
7,US-7734945-B1,Automated recovery of unbootable systems,20100608
8,US-7324671-B2,System and method for multi-view face detection,20080129
9,US-7478326-B2,Window information switching system,20090113


In [7]:
#Note on grant date- GBQ Patents stores the data as an int, but not an easily dealt-with int.
#here is a handy function to go to a string representation that is more readable and importable to e.g. Excel
repl = lambda m: m.group(0)[:4]+"-"+m.group(0)[4:6]+"-"+m.group(0)[6:]
df_patent_data['grant_date'] = df_patent_data['grant_date'].astype(str).str.replace("(\d{8})", repl)
display(df_patent_data.head(10))

,publication_number,title,grant_date
0,US-2006218651-A1,"Trusted infrastructure support systems, method...",0
1,US-2004123129-A1,"Trusted infrastructure support systems, method...",0
2,US-2003097331-A1,Systems for financial and electronic commerce,0
3,US-2003004827-A1,Payment system,0
4,US-2005197957-A1,Parcel manager for distributed electronic bill...,0
5,US-8140567-B2,Measuring entity extraction complexity,2012-03-20
6,US-7734631-B2,Associating information with an electronic doc...,2010-06-08
7,US-7734945-B1,Automated recovery of unbootable systems,2010-06-08
8,US-7324671-B2,System and method for multi-view face detection,2008-01-29
9,US-7478326-B2,Window information switching system,2009-01-13
